In [182]:
import numpy as np

In [183]:
def parse_camera_file(file_path):
    """
    Parse our camera format.

    The format is (*.txt):
    
    4x4 matrix (camera extrinsic)
    space
    3x3 matrix (camera intrinsic)

    focal is extracted from the intrinsc matrix
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    camera_extrinsic = []
    for x in lines[0:4]:
        camera_extrinsic += [float(y) for y in x.split()]
    camera_extrinsic = np.array(camera_extrinsic).reshape(4, 4)

    camera_intrinsic = []
    for x in lines[5:8]:
        camera_intrinsic += [float(y) for y in x.split()]
    camera_intrinsic = np.array(camera_intrinsic).reshape(3, 3)

    focal = camera_intrinsic[0, 0]
    # print(camera_extrinsic, camera_intrinsic, focal)
    return camera_extrinsic, camera_intrinsic, focal

In [184]:
import collections
import os
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec

class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)

In [185]:
from natsort import natsorted
from scipy.spatial.transform import Rotation

def sim_to_colmap(extrinsics):
    R = extrinsics[:3, :3]
    t = extrinsics[:3, 3]

    # Invert the rotation matrix (transpose, since it's orthogonal)
    R_inv = R.T
    # R_inv = np.linalg.inv(R)

    # Compute the new translation vector
    # t_inv = -np.dot(R_inv, t)
    
    R_flip = Rotation.from_euler('z', 180, degrees=True).as_matrix()
    R_inv = R_flip @ R_inv
    
    t_inv = -R_inv @ t

    # R_flip = Rotation.from_euler('z', 180, degrees=True).as_matrix()
    # R_inv = R_flip @ R_inv

    # Form the inverted transformation matrix
    extrinsics = np.eye(4)
    # print(extrinsics)
    extrinsics[:3, :3] = R_inv
    extrinsics[:3, 3] = t_inv
    
    return extrinsics

# def sim_to_colmap2(extrinsics):
#     extr = np.linalg.inv(extrinsics)
#     R = extr[:3, :3]
#     t = extr[:3, 3]

#     R_flip = Rotation.from_euler('z', 180, degrees=True).as_matrix()
#     R = R_flip @ R
#     # print(Rotation.from_euler('z', 180, degrees=True).as_matrix())
    
#     extrinsics = np.eye(4)
#     # print(extrinsics)
#     extrinsics[:3, :3] = R
#     extrinsics[:3, 3] = t
    
#     return extrinsics

In [186]:

path = 'data/slide_block_to_color_target_64_2/'
images_txt = open(f"{path}/sparse/0/images.txt", "w")
camera_txt = open(f"{path}/sparse/0/cameras.txt", "w")
pose_txt_files = natsorted(os.listdir(path + 'poses/'))

for txt_file in pose_txt_files:
    file = os.path.join(path + 'poses/',txt_file)
    filename = txt_file.split('.')[0]
    # print(file)
    # images = {}
    # with open(path, "r") as fid:

    camera_extrinsic, camera_intrinsic, focal = parse_camera_file(file)
    # print(camera_extrinsic)
    camera_extrinsic = sim_to_colmap(camera_extrinsic)
    # print(camera_extrinsic)
    rotation, translation = camera_extrinsic[:3,:3], camera_extrinsic[:3,3]
    # print(rotation, translation)
    
    
    # R_coppeliasim = camera_extrinsic[:3, :3]
    # T_coppeliasim = camera_extrinsic[:3, 3]

    # # Invert the rotation matrix (transpose)
    # R_colmap = R_coppeliasim.T

    # # Invert the translation vector
    # tvec = -R_colmap @ T_coppeliasim
    # # tvec = T_coppeliasim
    # # R_flip = np.array([[1, 0, 0],
    # #                    [0, -1, 0],
    # #                    [0, 0, -1]])  # 180-degree rotation around the X-axis
    
    # R_flip = Rotation.from_euler('z', 180, degrees=True).as_matrix()
    # R_colmap = R_flip @ R_colmap
    
    
    
    # Convert rotation matrix to quaternion
    rotation = Rotation.from_matrix(rotation)
    q_colmap = rotation.as_quat()  # Output as [qx, qy, qz, qw], COLMAP expects [qw, qx, qy, qz]

    # Reorder quaternion to COLMAP convention [qw, qx, qy, qz]
    qvec = np.roll(q_colmap, 1)
    
    # Return the quaternion and translation vector in COLMAP format

    # rotation = rotation.T
    # print(rotation,translation)
    # r_90 = np.array([[1, 0, 0], 
    #                  [0, -1, 0], 
    #                  [0, 0, 1]])

    # quaternions = Rotation.from_matrix(rotation) 
    image_id = filename
    # qvec = rotmat2qvec(rotation) # w x y z
    # # qvec = quaternions.as_quat() # x y z w
    # # print(quaternions.as_quat())
    # # print(qvec2rotmat(qvec))
    # # print(qvec)
    # tvec = translation
    # tvec = np.matmul(rotation, r_90)
    
    
    
    # camera_id = int(elems[8])
    camera_id = 1
    # image_name = elems[9]
    image_name = f'{filename}.png'
    # print(filename, qvec)
    
    

    
    qvec_str = " ".join([str(qvec[0]), str(qvec[1]), str(qvec[2]), str(qvec[3])])
    tvec_str = " ".join([str(translation[0]), str(translation[1]), str(translation[2])])
    data = " ".join([str(image_id), qvec_str, tvec_str, str(camera_id),str(image_name),'\n'])
    images_txt.write(data)
camera_intrinsic
fx,_,cx,_,fy,cy,_,_,_= camera_intrinsic.flatten()
data = " ".join([str(camera_id), 'PINHOLE','128', '128', str(-fx), str(-fy), str(cx), str(cy)])
camera_txt.write(data)

images_txt.close()
camera_txt.close()